In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In the previous notebook we explained and showed how we can preprocess data with multi-hot columns with NVTabular, and train an TF MLP model using NVTabular KerasSequenceLoader. We learned how to save a workflow, a trained TF model, and the ensemble model. In this notebook, we will show an example request script sent to triton inference server

- to transform new/streaming data with NVTabular library
- to deploy the end-to-end pipeline to generate prediction results for new data from trained TF model

## Getting Started

In [2]:
# External dependencies
import os
from os import path                
import time
import gc

import nvtabular
import cudf 
from tritonclient.utils import *
import tritonclient.grpc as grpcclient
import nvtabular.inference.triton as nvt_triton

We define our base directory, containing the data.

In [3]:
# path to store raw and preprocessed data
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', '/model/data/')

Let's deactivate the warnings before sending requests.

In [14]:
import warnings
warnings.filterwarnings('ignore')

/opt/conda/envs/rapids/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Verify Triton Is Running Correctly

Use Triton’s ready endpoint to verify that the server and the models are ready for inference. Replace localhost with your host ip address.

In [9]:
import tritonhttpclient
try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


/opt/conda/envs/rapids/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
triton_client.is_server_live()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


/opt/conda/envs/rapids/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


True

The HTTP request returns status 200 if Triton is ready and non-200 if it is not ready.

## Send request to Triton IS to transform raw dataset

A minimal model repository for a TensorFlow SavedModel model is:
```
  <model-repository-path>/<model-name>/
      config.pbtxt
      1/
        model.savedmodel/
           <saved-model files>
```
Let's check out our model repository layout. You can install tree library with apt-get install tree, and then run tree /model/models/ to print out the model repository layout as below:
               
```
├── movielens_mh
│   ├── 1
│   └── config.pbtxt
├── movielens_mh_nvt
│   ├── 1
│   │   ├── model.py
│   │   ├── __pycache__
│   │   │   └── model.cpython-38.pyc
│   │   └── workflow
│   │       ├── categories
│   │       │   ├── unique.genres.parquet
│   │       │   ├── unique.movieId.parquet
│   │       │   └── unique.userId.parquet
│   │       ├── metadata.json
│   │       └── workflow.pkl
│   └── config.pbtxt
└── movielens_mh_tf
    ├── 1
    │   └── model.savedmodel
    │       ├── assets
    │       ├── saved_model.pb
    │       └── variables
    │           ├── variables.data-00000-of-00001
    │           └── variables.index
    └── config.pbtxt
```
You can see that we have a `config.pbtxt` file. Each model in a model repository must include a model configuration that provides required and optional information about the model. Typically, this configuration is provided in a `config.pbtxt` file specified as [ModelConfig protobuf](https://github.com/triton-inference-server/server/blob/r20.12/src/core/model_config.proto).

Let's read the raw validation set, and send 3 rows of `userid` and `movieId` as input to the saved NVTabular model.

In [11]:
df_valid = cudf.read_parquet(INPUT_DATA_DIR + 'valid.parquet')
df_valid.head()

/opt/conda/envs/rapids/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,userId,movieId,rating
15347762,99476,104374,3.5
16647840,107979,2634,4.0
23915192,155372,1614,3.0
10052313,65225,7153,4.0
12214125,79161,500,5.0


In [15]:
df = cudf.DataFrame({"userId": [99476, 107979, 155372], "movieId": [104374, 2634, 1614]})
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], df, grpcclient.InferInput)

outputs = [
    grpcclient.InferRequestedOutput(col)
    for col in ["userId", "movieId", "genres__nnzs", "genres__values"]
]

MODEL_NAME_NVT = os.environ.get('MODEL_NAME_NVT', 'movielens_mh_nvt')

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs, request_id="1", outputs=outputs)

for col in ["userId", "movieId", "genres__nnzs", "genres__values"]:
    print(col, response.as_numpy(col), response.as_numpy(col).shape)

userId [[ 99476]
 [107979]
 [155372]] (3, 1)
movieId [[19997]
 [ 2543]
 [ 1557]] (3, 1)
genres__nnzs [[3]
 [1]
 [1]] (3, 1)
genres__values [[ 9]
 [10]
 [16]
 [12]
 [ 6]] (5, 1)


You might notice that we don't need to send the genres column as an input. The reason for that is the nvt model will look up the genres for each movie as part of the `JoinExternal` op it applies. Also notice that when creating the request for the `movielens_mh_nvt` model, we return 2 columns (values and nnzs) for the `genres` column rather than 1.

## END-2-END INFERENCE PIPELINE

We will do the same, but this time we directly read in first 3 rows of the the raw `valid.parquet` file with cuDF.

In [16]:
# read in the workflow (to get input/output schema to call triton with)
batch = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=['userId', 'movieId'])

print("raw data:\n", batch, "\n")

# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("dense_3")]

MODEL_NAME_ENSEMBLE = os.environ.get('MODEL_NAME_ENSEMBLE', 'movielens_mh')

# build a client to connect to our server. 
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1",outputs=outputs)

print("predicted softmax result:\n", response.as_numpy('dense_3'))


raw data:
           userId  movieId
15347762   99476   104374
16647840  107979     2634
23915192  155372     1614 

predicted softmax result:
 [[0.64413506]
 [0.5824011 ]
 [0.56059355]]


In [17]:
del df_valid
gc.collect()

148